In [208]:
import pandas as pd
import numpy as np

### Import dataset

In [ ]:
df = pd.read_csv("data/raw/michelin_by_Jerry_Ng.csv")

### Exploratory data analysis

In [210]:
df.head(3)

,Name,Address,Location,Price,Cuisine,Longitude,Latitude,PhoneNumber,Url,WebsiteUrl,Award,GreenStar,FacilitiesAndServices,Description
0,Taïrroir,"6F, 299 Lequn 3rd Road, Zhongshan District, Ta...","Taipei, Taiwan",$$$$,Taiwanese contemporary,121.559303,25.082896,8.862850e+11,https://guide.michelin.com/en/taipei-region/ta...,https://www.tairroir.com/,3 Stars,0,"Air conditioning,Wheelchair access","A portmanteau of Taiwan and terroir, Taïrroir ..."
1,JL Studio,"2F, 689, Section 4, Yifeng Road, Nantun Distri...","Taichung, Taiwan",$$$$,"Singaporean, Contemporary",120.628520,24.150486,8.864238e+11,https://guide.michelin.com/en/taichung-region/...,https://jlstudiotw.com,3 Stars,0,"Air conditioning,Car park,Wheelchair access","JL stands for Jimmy Lim, a Singaporean chef wh..."
2,Le Palais,"17F, Palais de Chine Hotel, 3, Section 1, Chen...","Taipei, Taiwan",$$$$,Cantonese,121.516889,25.049163,8.862218e+11,https://guide.michelin.com/en/taipei-region/ta...,https://www.palaisdechinehotel.com/pdc-en/page...,3 Stars,0,"Air conditioning,Car park,Wheelchair access","After the departure of the Macanese chef, the ..."


In [211]:
df.shape

(15520, 14)

In [212]:
df.info()  # data type

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15520 entries, 0 to 15519
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Name                   15520 non-null  object 
 1   Address                15520 non-null  object 
 2   Location               15520 non-null  object 
 3   Price                  15519 non-null  object 
 4   Cuisine                15519 non-null  object 
 5   Longitude              15520 non-null  float64
 6   Latitude               15520 non-null  float64
 7   PhoneNumber            15182 non-null  float64
 8   Url                    15520 non-null  object 
 9   WebsiteUrl             13245 non-null  object 
 10  Award                  15520 non-null  object 
 11  GreenStar              15520 non-null  int64  
 12  FacilitiesAndServices  14665 non-null  object 
 13  Description            15519 non-null  object 
dtypes: float64(3), int64(1), object(10)
memory usage: 1.7+

In [213]:
df[
    "Award"
].value_counts()

Award
Selected Restaurants    10484
Bib Gourmand             3238
1 Star                   1160
2 Stars                   493
3 Stars                   145
Name: count, dtype: int64

In [214]:
df[
    "GreenStar"
].value_counts()

GreenStar
0    15279
1      241
Name: count, dtype: int64

In [215]:
df[
    "Name"
].nunique()

14980

In [216]:
df[
    "Cuisine"
].value_counts().head(15)

Cuisine
Modern Cuisine           2425
Traditional Cuisine       760
Creative                  461
Japanese                  458
Italian                   404
Contemporary              400
Seafood                   353
French                    319
Street Food               308
Modern British            252
Mediterranean Cuisine     210
Classic Cuisine           206
Farm to table             195
Thai                      171
Cantonese                 165
Name: count, dtype: int64

In [217]:
df[
    "Price"
].value_counts()

Price
€€      4675
€€€     2773
€€€€    1278
$$      1155
$$$      838
$$$$     625
¥¥¥      573
€        553
$        495
££       451
¥¥       414
£££      301
฿฿       210
¥        198
¥¥¥¥     156
฿        128
££££     116
₩         87
₫         76
฿฿฿       71
₩₩        57
₫₫        48
₺₺        46
₩₩₩       32
₩₩₩₩      31
฿฿฿฿      29
₫₫₫       25
₺₺₺       20
₫₫₫₫      15
₺₺₺₺      15
£         14
₺         14
Name: count, dtype: int64

### Data cleaning

In [218]:
# Drop columns
df = df.drop(columns=['PhoneNumber', 'Url', 'WebsiteUrl', 'FacilitiesAndServices', 'Description'])

In [219]:
# Remove rows with NA values ​​only from columns "Price" and "Cuisine"
df = df.dropna(subset=["Price", "Cuisine"])

In [220]:
# Insert new column with price in dollars only
def convert_to_dollars(price):
    return "$" * len(price)

df["Price_dollar"] = df["Price"].apply(convert_to_dollars)

In [221]:
# Change 0 and 1 in the "GreenStar" column to Yes and No
df["GreenStar"] = df["GreenStar"].replace({0: "No", 1: "Yes"})

In [222]:
# Remove all cuisine types after comma
df["Cuisine"] = df["Cuisine"].str.split(",").str[0].str.strip()

In [223]:
df[
    "Cuisine"
].nunique()

259

### Create columns

In [224]:
# Split "Location" column
df[['City', 'Country']] = df['Location'].str.split(',', n=1, expand=True).apply(lambda x: x.str.strip())

In [225]:
# Identify rows with NA
df[df['Country'].isna()][['City', 'Country']].drop_duplicates()


,City,Country
9,Singapore,None
67,Hong Kong,None
70,Macau,None
163,Dubai,None
3314,Luxembourg,None
3811,Abu Dhabi,None


In [226]:
location = {"Singapore": "Republic of Singapore",
              "Hong Kong": "Hong Kong SAR China",
              "Macau": "Macau SAR China",
              "Dubai": "United Arab Emirates",
              "Abu Dhabi": "United Arab Emirates",
              "Luxembourg": "Luxembourg",
              }

# Replace values ​​in the 'Country' column where they are NA
df['Country'] = df['Country'].fillna(df['City'].map(location))

In [227]:
# Replaces "Luxembourg" with NA in 'City' where 'City' and 'Country' are both "Luxembourg"
df.loc[df[['City', 'Country']].eq("Luxembourg").all(axis=1), 'City'] = np.nan


In [228]:
award_info_en = {"3 Stars": "Exceptional cuisine",
              "2 Stars": "Excellent cooking",
              "1 Star": "High quality cooking",
              "Bib Gourmand": "Good quality, good value cooking",
              "Selected Restaurants": "Good cooking",
              }

df["Award_info_en"] = df["Award"].map(award_info_en)

In [229]:
award_info_br = {"3 Stars": "Cozinha excepcional",
              "2 Stars": "Cozinha excelente",
              "1 Star": "Cozinha requintada",
              "Bib Gourmand": "Melhor relação qualidade-preço ",
              "Selected Restaurants": "Boa cozinha",
              }

df["Award_info_br"] = df["Award"].map(award_info_br)

### Translate columns

#### Country

In [230]:
unique_award = df["Country"].unique()
print(unique_award)

['Taiwan' 'USA' 'Republic of Singapore' 'Slovenia' 'Sweden' 'Denmark'
 'Norway' 'Japan' 'Germany' 'France' 'Hong Kong SAR China'
 'Macau SAR China' 'Belgium' 'United Kingdom' 'Spain' 'China Mainland'
 'Italy' 'Switzerland' 'Netherlands' 'Austria' 'United Arab Emirates'
 'Croatia' 'Poland' 'Finland' 'Estonia' 'Brazil' 'Mexico' 'Malta'
 'Portugal' 'Luxembourg' 'South Korea' 'Ireland' 'Thailand' 'Argentina'
 'Malaysia' 'Türkiye' 'Greece' 'Canada' 'Hungary' 'Vietnam' 'Lithuania'
 'Iceland' 'Serbia' 'Latvia' 'Czech Republic' 'Czechia' 'Andorra']


In [231]:
country_br = {
    "Taiwan": "Taiwan",
    "USA": "Estados Unidos",
    "Republic of Singapore": "República de Singapura",
    "Slovenia": "Eslovênia",
    "Sweden": "Suécia",
    "Denmark": "Dinamarca",
    "Norway": "Noruega",
    "Japan": "Japão",
    "Germany": "Alemanha",
    "France": "França",
    "Hong Kong SAR China": "Hong Kong (RAE China)",
    "Macau SAR China": "Macau (RAE China)",
    "Belgium": "Bélgica",
    "United Kingdom": "Reino Unido",
    "Spain": "Espanha",
    "China Mainland": "China Continental",
    "Italy": "Itália",
    "Switzerland": "Suíça",
    "Netherlands": "Países Baixos",
    "Austria": "Áustria",
    "United Arab Emirates": "Emirados Árabes Unidos",
    "Croatia": "Croácia",
    "Poland": "Polônia",
    "Finland": "Finlândia",
    "Estonia": "Estônia",
    "Brazil": "Brasil",
    "Mexico": "México",
    "Malta": "Malta",
    "Portugal": "Portugal",
    "Luxembourg": "Luxemburgo",
    "South Korea": "Coreia do Sul",
    "Ireland": "Irlanda",
    "Thailand": "Tailândia",
    "Argentina": "Argentina",
    "Malaysia": "Malásia",
    "Türkiye": "Turquia",
    "Greece": "Grécia",
    "Canada": "Canadá",
    "Hungary": "Hungria",
    "Vietnam": "Vietnã",
    "Lithuania": "Lituânia",
    "Iceland": "Islândia",
    "Serbia": "Sérvia",
    "Latvia": "Letônia",
    "Czech Republic": "República Tcheca",
    "Czechia": "Tchéquia",
    "Andorra": "Andorra"
}

# Creste column "Country_br"
df["Country_br"] = df["Country"].map(country_br)

In [232]:
df["Location_br"] = df["City"] + ", " + df["Country_br"]

#### Cuisine

In [233]:
unique_cuisines = df["Cuisine"].unique()
print(unique_cuisines)

['Taiwanese contemporary' 'Singaporean' 'Cantonese' 'Contemporary' 'Asian'
 'European Contemporary' 'French Contemporary' 'French' 'Creative'
 'Modern Cuisine' 'Japanese' 'Innovative' 'Classic French' 'Modern French'
 'Seafood' 'Classic Cuisine' 'Mediterranean Cuisine' 'Italian' 'Sushi'
 'Creative French' 'Creative British' 'Modern British' 'Chinese'
 'Italian Contemporary' 'American' 'Vegan' 'Chao Zhou' 'Taizhou'
 'Vegetarian' 'Spanish Contemporary' 'Asian Contemporary' 'Tempura'
 'Mexican' 'Californian' 'Indian' 'British Contemporary' 'Scandinavian'
 'Hunanese' 'Huaiyang' 'Traditional Cuisine' 'Asian Influences' 'Korean'
 'Turkish' 'Traditional British' 'Thai contemporary' 'German'
 'Southern Thai' 'Thai' 'Spanish' 'Malaysian' 'Country cooking'
 'International' 'Piedmontese' 'Beijing Cuisine' 'Shanghainese' 'Sharing'
 'Farm to table' 'Japanese Contemporary' 'Sichuan' 'Organic'
 'Latin American' 'Taiwanese' 'Hang Zhou' 'Barbecue' 'Steakhouse'
 'Portuguese' 'Indian Vegetarian' 'Vietnam

In [234]:
cuisine_br = {
    "Taiwanese contemporary": "Taiwanesa contemporânea",
    "Singaporean": "Singapurense",
    "Cantonese": "Cantonês",
    "Contemporary": "Contemporânea",
    "Asian": "Asiática",
    "European Contemporary": "Europeia Contemporânea",
    "French Contemporary": "Francesa Contemporânea",
    "French": "Francesa",
    "Creative": "Criativa",
    "Modern Cuisine": "Culinária Moderna",
    "Japanese": "Japonesa",
    "Innovative": "Inovadora",
    "Classic French": "Francesa Clássica",
    "Modern French": "Francesa Moderna",
    "Seafood": "Frutos do Mar",
    "Classic Cuisine": "Culinária Clássica",
    "Mediterranean Cuisine": "Culinária Mediterrânea",
    "Italian": "Italiana",
    "Sushi": "Sushi",
    "Creative French": "Francesa Criativa",
    "Creative British": "Britânica Criativa",
    "Modern British": "Britânica Moderna",
    "Chinese": "Chinesa",
    "Italian Contemporary": "Italiana Contemporânea",
    "American": "Americana",
    "Vegan": "Vegana",
    "Chao Zhou": "Chao Zhou",
    "Taizhou": "Taizhou",
    "Vegetarian": "Vegetariana",
    "Spanish Contemporary": "Espanhola Contemporânea",
    "Asian Contemporary": "Asiática Contemporânea",
    "Tempura": "Tempura",
    "Mexican": "Mexicana",
    "Californian": "Californiana",
    "Indian": "Indiana",
    "British Contemporary": "Britânica Contemporânea",
    "Scandinavian": "Escandinava",
    "Hunanese": "Hunanesa",
    "Huaiyang": "Huaiyang",
    "Traditional Cuisine": "Culinária Tradicional",
    "Asian Influences": "Influências Asiáticas",
    "Korean": "Coreana",
    "Turkish": "Turca",
    "Traditional British": "Britânica Tradicional",
    "Thai contemporary": "Tailandesa contemporânea",
    "German": "Alemã",
    "Southern Thai": "Tailandesa do Sul",
    "Thai": "Tailandesa",
    "Spanish": "Espanhola",
    "Malaysian": "Malaia",
    "Country cooking": "Culinária Caseira",
    "International": "Internacional",
    "Piedmontese": "Piemontesa",
    "Beijing Cuisine": "Culinária de Pequim",
    "Shanghainese": "Xangainesa",
    "Sharing": "Compartilhada",
    "Farm to table": "Da Fazenda à Mesa",
    "Japanese Contemporary": "Japonesa Contemporânea",
    "Sichuan": "Sichuan",
    "Organic": "Orgânica",
    "Latin American": "Latino-Americana",
    "Taiwanese": "Taiwanesa",
    "Hang Zhou": "Hang Zhou",
    "Barbecue": "Churrasco",
    "Steakhouse": "Churrascaria",
    "Portuguese": "Portuguesa",
    "Indian Vegetarian": "Indiana Vegetariana",
    "Vietnamese Contemporary": "Vietnamita Contemporânea",
    "Teppanyaki": "Teppanyaki",
    "Vietnamese": "Vietnamita",
    "Teochew": "Teochew",
    "Korean Contemporary": "Coreana Contemporânea",
    "Street Food": "Comida de Rua",
    "Peranakan": "Peranakan",
    "European": "Europeia",
    "Dim Sum": "Dim Sum",
    "Fujian": "Fujian",
    "Grills": "Grelhados",
    "Norwegian": "Norueguesa",
    "Zhejiang": "Zhejiang",
    "Chinese Contemporary": "Chinesa Contemporânea",
    "Peruvian": "Peruana",
    "Colombian": "Colombiana",
    "Fusion": "Fusão",
    "Fugu / Pufferfish": "Fugu / Baiacu",
    "Yakitori": "Yakitori",
    "Seasonal Cuisine": "Culinária Sazonal",
    "Austrian": "Austríaca",
    "Israeli": "Israelense",
    "Regional Cuisine": "Culinária Regional",
    "Provençal": "Provençal",
    "Greek": "Grega",
    "Noodles": "Macarrão",
    "Small eats": "Pequenos Pratos",
    "Dumplings": "Bolinho de Massa",
    "Indonesian": "Indonésia",
    "Congee": "Congee",
    "Jiangzhe": "Jiangzhe",
    "Hakkanese": "Hakkanesa",
    "Pizza": "Pizza",
    "Filipino": "Filipina",
    "Southern": "Sulista",
    "Gastropub": "Gastropub",
    "Deli": "Delicatessen",
    "Italian-American": "Italo-Americana",
    "Middle Eastern": "Oriente Médio",
    "Burmese": "Birmanesa",
    "Pakistani": "Paquistanesa",
    "Ethiopian": "Etíope",
    "South East Asian": "Sudeste Asiático",
    "Lebanese": "Libanesa",
    "Asian and Western": "Asiática e Ocidental",
    "Persian": "Persa",
    "Balkan": "Balcânica",
    "Emirati Cuisine": "Culinária Emiradense",
    "Meats and Grills": "Carnes e Grelhados",
    "South American": "Sul-Americana",
    "Regional European": "Europeia Regional",
    "Xibei": "Xibei",
    "Noodles and Congee": "Macarrão e Congee",
    "Swedish": "Sueca",
    "Smørrebrød": "Smørrebrød",
    "Danish": "Dinamarquesa",
    "Brazilian": "Brasileira",
    "Unagi / Freshwater Eel": "Unagi / Enguia de Água Doce",
    "Soba": "Soba",
    "Kushiage": "Kushiage",
    "Ramen": "Ramen",
    "Ningbo": "Ningbo",
    "Cuban": "Cubana",
    "Nepali": "Nepalesa",
    "Bakery": "Padaria",
    "Izakaya": "Izakaya",
    "Tonkatsu": "Tonkatsu",
    "Udon": "Udon",
    "Oden": "Oden",
    "Yoshoku": "Yoshoku",
    "Shojin": "Shojin",
    "Obanzai": "Obanzai",
    "World Cuisine": "Culinária Mundial",
    "Alpine": "Alpina",
    "Home Cooking": "Culinária Caseira",
    "Savoyard": "Savoiana",
    "Corsican": "Corsa",
    "Alsatian": "Alsaciana",
    "Catalan": "Catalã",
    "Breton": "Bretã",
    "Cantonese Roast Meats": "Carnes Assadas Cantonês",
    "Chiu Chow": "Chiu Chow",
    "Macanese": "Macaense",
    "Singaporean and Malaysian": "Singapurense e Malaia",
    "Xinjiang": "Xinjiang",
    "Shun Tak": "Shun Tak",
    "Belgian": "Belga",
    "Cuisine from South West France": "Culinária do Sudoeste da França",
    "Dwaeji-gukbap": "Dwaeji-gukbap",
    "Gomtang": "Gomtang",
    "Naengmyeon": "Naengmyeon",
    "Chueotang": "Chueotang",
    "Gejang": "Gejang",
    "Yukhoe": "Yukhoe",
    "Bulgogi": "Bulgogi",
    "Memil-guksu": "Memil-guksu",
    "Mandu": "Mandu",
    "Seolleongtang": "Seolleongtang",
    "Dubu": "Dubu",
    "Jokbal": "Jokbal",
    "Doganitang": "Doganitang",
    "Kalguksu": "Kalguksu",
    "Sujebi": "Sujebi",
    "South Indian": "Sul da Índia",
    "North African": "Norte-Africana",
    "Creole": "Crioula",
    "Scottish": "Escocesa",
    "South African": "Sul-Africana",
    "Sri Lankan": "Cingalesa",
    "Isan": "Isan",
    "Thai-Chinese": "Tailandesa-Chinesa",
    "Northern Thai": "Norte da Tailândia",
    "Onigiri": "Onigiri",
    "Pork": "Porco",
    "Galician": "Galega",
    "Rice Dishes": "Pratos de Arroz",
    "Basque": "Basca",
    "Andalusian": "Andaluza",
    "Argentinian": "Argentina",
    "Tuscan": "Toscana",
    "Venetian": "Veneziana",
    "Emilian": "Emiliana",
    "Cuisine from Valtellina": "Culinária de Valtellina",
    "Sicilian": "Siciliana",
    "Cuisine from Abruzzo": "Culinária de Abruzzo",
    "Sardinian": "Sarda",
    "Cuisine from the Marches": "Culinária das Marcas",
    "Lombardian": "Lombarda",
    "Apulian": "Apuliana",
    "Mantuan": "Mantovana",
    "Roman": "Romana",
    "Cuisine from Basilicata": "Culinária da Basilicata",
    "Umbrian": "Umbra",
    "Ligurian": "Liguriana",
    "South Tyrolean": "Sul-Tirolense",
    "Cuisine from Lazio": "Culinária do Lácio",
    "Campanian": "Campana",
    "Friulian": "Friulana",
    "Cuisine from Romagna": "Culinária da Romagna",
    "Calabrian": "Calabresa",
    "Cuisine from the Aosta Valley": "Culinária do Vale de Aosta",
    "Caribbean": "Caribenha",
    "Afghan": "Afegã",
    "Tibetan": "Tibetana",
    "Tex-Mex": "Tex-Mex",
    "Central Asian": "Centro-Asiática",
    "Croatian": "Croata",
    "Czech": "Tcheca",
    "Polish": "Polonesa",
    "Hotpot": "Hotpot",
    "Shandong": "Shandong",
    "Hubei": "Hubei",
    "Swiss": "Suíça",
    "Hungarian": "Húngara",
    "Yunnanese": "Yunnanesa",
    "Shabu-shabu": "Shabu-shabu",
    "Sukiyaki": "Sukiyaki",
    "Armenian": "Armênia",
    "Puerto Rican": "Porto-Riquenha",
    "Moroccan": "Marroquina",
    "Meats and Seafood": "Carnes e Frutos do Mar",
    "Asturian": "Asturiana",
    "Lyonnaise": "Lionesa",
    "American Contemporary": "Americana Contemporânea",
    "Hainanese": "Hainanesa",
    "Finnish": "Finlandesa",
    "Italian and Japanese": "Italiana e Japonesa",
    "Chicken Specialities": "Especialidades de Frango",
    "Beef": "Carne de Vaca",
    "Eastern European": "Europa Oriental",
    "Lao": "Laosiana",
    "Duck Specialities": "Especialidades de Pato",
    "Bavarian": "Bávara",
    "African": "Africana",
    "Swabian": "Suábia",
    "Burgundian": "Burgúndia",
    "Cheese": "Queijo",
    "Australian Contemporary": "Australiana Contemporânea",
    "Flemish": "Flamenga",
    "Irish": "Irlandesa",
    "Castilian": "Castelhana",
    "Lamb Specialities": "Especialidades de Cordeiro",
    "Japanese Steakhouse": "Churrascaria Japonesa",
    "Hui Cuisine": "Culinária Hui",
    "Milanese": "Milanesa",
    "Russian": "Russa",
    "English": "Inglesa",
    "Jamaican": "Jamaicana",
    "Venezuelan": "Venezuelana",
    "Hawaiian": "Havaiana",
    "North American": "Norte-Americana",
    "Egyptian": "Egípcia",
}

# Creste column "Cuisine_br"
df["Cuisine_br"] = df["Cuisine"].map(cuisine_br)

#### Price

In [235]:
unique_award = df["Award"].unique()
print(unique_award)

['3 Stars' '2 Stars' '1 Star' 'Bib Gourmand' 'Selected Restaurants']


In [236]:
award_br = {"3 Stars": "3 Estrelas",
              "2 Stars": "2 Estrelas",
              "1 Star": "1 Estrela",
              "Bib Gourmand": "Bib Gourmand",
              "Selected Restaurants": "Restaurante selecionado",
              }

# Creste column "Award_br"
df["Award_br"] = df["Award"].map(award_br)

### Add scraped images

In [ ]:
df_scrap = pd.read_csv("data/scraper/images_url.csv")

In [238]:
df_final = pd.merge(df, df_scrap, on='Address', suffixes=('', '_drop')).drop(columns=['Name_drop'])

### Save dataset

In [239]:
# Drop column "PhoneNumber"
df_final = df_final.drop(columns=['City', 'Country', 'Country_br'])

In [241]:
# Convert all columns in the list to type "category"
columns_category = ["Location", "Price", "Price_dollar", "Cuisine", "Cuisine_br", "Award", "Award_br", "Location_br"]

for column in columns_category:
    df_final[column] = df_final[column].astype("category")

In [242]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15498 entries, 0 to 15497
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Name           15498 non-null  object  
 1   Address        15498 non-null  object  
 2   Location       15498 non-null  category
 3   Price          15498 non-null  category
 4   Cuisine        15498 non-null  category
 5   Longitude      15498 non-null  float64 
 6   Latitude       15498 non-null  float64 
 7   Award          15498 non-null  category
 8   GreenStar      15498 non-null  object  
 9   Price_dollar   15498 non-null  category
 10  Award_info_en  15498 non-null  object  
 11  Award_info_br  15498 non-null  object  
 12  Location_br    15480 non-null  category
 13  Cuisine_br     15498 non-null  category
 14  Award_br       15498 non-null  category
 15  ImageURL       15498 non-null  object  
dtypes: category(8), float64(2), object(6)
memory usage: 1.5+ MB


In [ ]:

df_final.to_parquet("data/ready/michelin.parquet")

In [244]:
df_final.head(3)

,Name,Address,Location,Price,Cuisine,Longitude,Latitude,Award,GreenStar,Price_dollar,Award_info_en,Award_info_br,Location_br,Cuisine_br,Award_br,ImageURL
0,Taïrroir,"6F, 299 Lequn 3rd Road, Zhongshan District, Ta...","Taipei, Taiwan",$$$$,Taiwanese contemporary,121.559303,25.082896,3 Stars,No,$$$$,Exceptional cuisine,Cozinha excepcional,"Taipei, Taiwan",Taiwanesa contemporânea,3 Estrelas,https://axwwgrkdco.cloudimg.io/v7/__gmpics3__/...
1,JL Studio,"2F, 689, Section 4, Yifeng Road, Nantun Distri...","Taichung, Taiwan",$$$$,Singaporean,120.628520,24.150486,3 Stars,No,$$$$,Exceptional cuisine,Cozinha excepcional,"Taichung, Taiwan",Singapurense,3 Estrelas,https://axwwgrkdco.cloudimg.io/v7/__gmpics3__/...
2,Le Palais,"17F, Palais de Chine Hotel, 3, Section 1, Chen...","Taipei, Taiwan",$$$$,Cantonese,121.516889,25.049163,3 Stars,No,$$$$,Exceptional cuisine,Cozinha excepcional,"Taipei, Taiwan",Cantonês,3 Estrelas,https://axwwgrkdco.cloudimg.io/v7/__gmpics3__/...
